## Querying a Redis index

Simple example on how to query content from a Redis VectorStore.

Requirements:
- A Redis cluster and Database where documents have been injected
- All information for connecting to the redis cluster and database, index name and schema file.

### Install Packages

In [ ]:
!pip install -qU langchain langchain-huggingface psycopg "psycopg[binary]" langchain-community sentence-transformers redis==5.2.0

### Base parameters, the Redis info

In [ ]:
import yaml

schema_name = "redis_schema.yaml"

with open(schema_name, 'r') as file:
    redis_schema = yaml.safe_load(file)

### Imports

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import redis
from langchain.vectorstores.redis import Redis

### Configure Embedding

from langchain_huggingface import HuggingFaceEmbeddings

# Specify a valid model name
model_name = "sentence-transformers/all-MiniLM-L6-v2"  # Example model

# Initialize HuggingFaceEmbeddings with the specified model
embeddings = HuggingFaceEmbeddings(model_name=model_name)


### Initialize the connection

In [ ]:
from redis import Redis as RedisClient
from langchain_community.vectorstores.redis import Redis

password="password" #Change this to the password of the redis instance
ragendpoint="redb-rag-instance.redis-rag.svc.cluster.local.local" # Double check the endpoint of the redis instance
port="14283" # Change this to the port of the redis instance
redis_url = "redis://default:"+password+"@"+ragendpoint+":"+port
index_name = "docs"


# Create Redis client explicitly
#redis_client = RedisClient.from_url(redis_url)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
rds = Redis.from_existing_index(
    embeddings,
    redis_url=redis_url,
    index_name=index_name,
    schema="redis_schema.yaml"
)

### Make a query to the index to verify sources

In [ ]:
query="How do you work with taints?"
results =rds.similarity_search(query, k=4, return_metadata=True)
for result in results:
    print(result.metadata['source'])

### Work with a retriever

In [ ]:
retriever = rds.as_retriever(search_type="similarity_distance_threshold", search_kwargs={"k": 4, "distance_threshold": 2})

In [ ]:
docs = retriever.get_relevant_documents(query)
docs